In [1]:
import pandas as pd
import numpy as np
import ast
import scipy 
import os
import re

In [2]:
import os
import pandas as pd
DIRNAME='./Feb16/'
dir_list=os.listdir(DIRNAME)
files_test=[]
for file in dir_list:
    if file.find('_test.xlsx')>0:
        files_test.append(file)

In [3]:
class Evaluation:
    def __init__(self,  oup_evaluation_file_name, oup_bucket_file, oup_bucket_grt_file):
        self.dir_name=DIRNAME
        self.df_oup=pd.DataFrame(columns=['language_pair', 'num_rows','pearson', 'spearmanr', 'kendalltau'])
        self.oup_evaluation_file_name=oup_evaluation_file_name
        self.df_oup_bucket=pd.DataFrame(columns=['language_pair', 'score_range' ,'num_rows','pearson', 'spearmanr', 'kendalltau'])
        self.oup_bucket_file=oup_bucket_file
        self.df_oup_bucket_grter=pd.DataFrame(columns=['language_pair', 'score_range' ,'num_rows','pearson', 'spearmanr', 'kendalltau'])
        self.oup_bucket_grt_file=oup_bucket_grt_file
        
    
    
    
    def score_correlation(self, file):
        print(file)
        df=pd.read_excel(self.dir_name+file)
        num_rows=len(df)
        language_pair=re.sub('_overlaps_test.xlsx', '', file )
        language_pair=re.sub('oup_test_', '', language_pair )
        #print(df)
        if df['second_prompt_output'].dtype=='int64':
            x=df['score'].to_numpy()
            y=df['second_prompt_output'].to_numpy()
            pearson=df['second_prompt_output'].corr(df['score'], method='pearson')
            spearmanr=df['second_prompt_output'].corr(df['score'], method='spearman')
            kendalltau=df['second_prompt_output'].corr(df['score'], method='kendall')

        else:
            pearson='Evaluation score Dtype not int'
            spearmanr='Evaluation score Dtype not int'
            kendalltau='Evaluation score Dtype not int'
            
        output_file_name=re.sub('.xlsx', '', file)+'_scoring'
        dir_files=os.listdir('./')
        
        if self.oup_evaluation_file_name in dir_files:
            df_oup=pd.read_excel(self.oup_evaluation_file_name)
            df_oup.reset_index(drop=True)
            df_oup.loc[len(df_oup)]=[language_pair,num_rows, pearson, spearmanr, kendalltau]
            df_oup.to_excel(self.oup_evaluation_file_name, index=False)
        else:
            self.df_oup.loc[len(self.df_oup)]=[language_pair,num_rows, pearson, spearmanr, kendalltau]
            self.df_oup.to_excel(self.oup_evaluation_file_name, index=False)
            
    
    def evaluation_buckets_lessthan(self, file):
        scores_bucket=[40,50,60,70,80,90,100]
        append_files=[]
        
        df=pd.read_excel(self.dir_name+file)
        num_rows=len(df)
        language_pair=re.sub('_overlaps_test.xlsx', '', file )
        language_pair=re.sub('oup_test_', '', language_pair )
        
        for score in scores_bucket:
            df_upd=df.loc[df['score']<score]
            num_rows=len(df_upd)
            pearson=df_upd['second_prompt_output'].corr(df_upd['score'], method='pearson')
            spearmanr=df_upd['second_prompt_output'].corr(df_upd['score'], method='spearman')
            kendalltau=df_upd['second_prompt_output'].corr(df_upd['score'], method='kendall')
            append_files.append([language_pair, 'score less than '+ str(score), num_rows,  pearson, spearmanr, kendalltau] )
            
        dir_files=os.listdir('./')
        if self.oup_bucket_file in dir_files:
            df_oup_bucket=pd.read_excel(self.oup_bucket_file)
            df_oup_bucket.reset_index(drop=True)
            i=len(df_oup_bucket)
            for row in append_files:
                df_oup_bucket.loc[i]=row
                i+=1
            df_oup_bucket.to_excel(self.oup_bucket_file, index=False)
        else:
            i=len(self.df_oup_bucket)
            for row in append_files:
                self.df_oup_bucket.loc[i]=row
                i+=1
            self.df_oup_bucket.to_excel(self.oup_bucket_file, index=False)
            
    def evaluation_buckets_grtethan(self, file):
        scores_bucket=[1, 40,50,60,70,80,90]
        append_files=[]
        
        df=pd.read_excel(self.dir_name+file)
        num_rows=len(df)
        language_pair=re.sub('_overlaps_test.xlsx', '', file )
        language_pair=re.sub('oup_test_', '', language_pair )
        
        for score in scores_bucket:
            df_upd=df.loc[df['score']>score]
            num_rows=len(df_upd)
            pearson=df_upd['second_prompt_output'].corr(df_upd['score'], method='pearson')
            spearmanr=df_upd['second_prompt_output'].corr(df_upd['score'], method='spearman')
            kendalltau=df_upd['second_prompt_output'].corr(df_upd['score'], method='kendall')
            append_files.append([language_pair, 'score greater than '+ str(score), num_rows,  pearson, spearmanr, kendalltau] )
            
        dir_files=os.listdir('./')
        if self.oup_bucket_grt_file in dir_files:
            df_oup_bucket_grter=pd.read_excel(self.oup_bucket_grt_file)
            df_oup_bucket_grter.reset_index(drop=True)
            i=len(df_oup_bucket_grter)
            for row in append_files:
                df_oup_bucket_grter.loc[i]=row
                i+=1
            df_oup_bucket_grter.to_excel(self.oup_bucket_grt_file, index=False)
        else:
            i=len(self.df_oup_bucket_grter)
            for row in append_files:
                self.df_oup_bucket_grter.loc[i]=row
                i+=1
            self.df_oup_bucket_grter.to_excel(self.oup_bucket_grt_file, index=False)


        

In [4]:
!rm Evaluation_correlation.xlsx
!rm Evaluation_buckets_correlation.xlsx
!rm Evaluation_buckets_grtthancorrelation.xlsx
evaluation=Evaluation('./Evaluation_correlation.xlsx', './Evaluation_buckets_lessthancorrelation.xlsx',  './Evaluation_buckets_grtthancorrelation.xlsx')
for file in files_test:
    evaluation.score_correlation(file)
    evaluation.evaluation_buckets_lessthan(file)
    evaluation.evaluation_buckets_grtethan(file)
    

/usr/bin/rmsafe: cannot remove 'Evaluation_buckets_correlation.xlsx': No such file or directory
oup_test_en-de_overlaps_test.xlsx
oup_test_en-mr_overlaps_test.xlsx
oup_test_en-zh_overlaps_test.xlsx
oup_test_et-en_overlaps_test.xlsx
oup_test_ne-en_overlaps_test.xlsx
oup_test_ro-en_overlaps_test.xlsx
oup_test_ru-en_overlaps_test.xlsx
oup_test_si-en_overlaps_test.xlsx


In [5]:
df_en_de_test=pd.read_excel(DIRNAME+'oup_test_en-de_overlaps_test.xlsx')
df_en_de_test

,src,mt,ref,tags,mt4word,score,z_score,error_words,unique_id,first_prompt,first_prompt_output,second_prompt,second_prompt_output
0,"On Chicago's next possession, Michael Bush fum...",Bei Chicagos nächster Besessenheit fummelte Mi...,Bei Chicagos nächstem Ballerwerb fummelte Mich...,OK OK BAD BAD OK OK OK BAD OK OK OK OK OK OK O...,Bei Chicagos nächster Besessenheit fummelte Mi...,89.333333,-0.163494,"nächster, Besessenheit, ,",0,GPT4 Correct User: There is an English to Germ...,1. Accuracy:\n - Machine translation: The ma...,GPT4 Correct User: There is an English to Germ...,85
1,Lucan repaid some of his creditors and purchas...,Lucan zahlte einige seiner Gläubiger zurück un...,Lucan zahlte einige seiner Gläubiger zurück un...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...,Lucan zahlte einige seiner Gläubiger zurück un...,80.666667,-0.413205,"sie, Veronikas, dekorieren",1,GPT4 Correct User: There is an English to Germ...,1. Accuracy:\n - Machine translation: Lucan ...,GPT4 Correct User: There is an English to Germ...,90
2,It centred on the orchestras of Chicago and Cl...,Im Mittelpunkt standen die Orchester von Chica...,Im Mittelpunkt standen die Orchester von Chica...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...,Im Mittelpunkt standen die Orchester von Chica...,97.000000,0.722348,NaN,2,GPT4 Correct User: There is an English to Germ...,1. Accuracy:\n - Machine translation: Im Mit...,GPT4 Correct User: There is an English to Germ...,100
3,"However, with a little brainstorming, addition...",Mit ein wenig Brainstorming können jedoch zusä...,Mit ein wenig Brainstorming können jedoch zusä...,OK OK OK OK OK OK OK OK BAD BAD OK OK,Mit ein wenig Brainstorming können jedoch zusä...,92.666667,0.577922,"zutage, treten",3,GPT4 Correct User: There is an English to Germ...,1. Accuracy:\n - Machine translation: The ma...,GPT4 Correct User: There is an English to Germ...,95
4,;Sensing Concrete Realistic Practical Experien...,; Konkrete realistische praktische Erfahrung z...,; Fühlen Konkret Realistisch Praktisch Experim...,OK BAD BAD BAD BAD BAD BAD BAD OK,; Konkrete realistische praktische Erfahrung z...,68.000000,-1.673285,"Konkrete, realistische, praktische, Erfahrung,...",4,GPT4 Correct User: There is an English to Germ...,1. Accuracy:\n - Machine translation: Konkre...,GPT4 Correct User: There is an English to Germ...,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,Colorful ceremonial costumes includes an elabo...,Bunte zeremonielle Kostüme umfassen eine aufwe...,Bunte zeremonielle Kostüme umfassen eine aufwe...,OK OK OK OK OK OK BAD BAD OK OK OK OK OK OK OK OK,Bunte zeremonielle Kostüme umfassen eine aufwe...,84.666667,-0.198707,"Perlen, Kopfbedeckung",887,GPT4 Correct User: There is an English to Germ...,1. Accuracy:\n - Machine translation: Bunte ...,GPT4 Correct User: There is an English to Germ...,95
888,It served as venues for 2004 Pekan Olahraga Na...,Es diente als Veranstaltungsort für 2004 Pekan...,Es diente als Veranstaltungsort für 2004 Pekan...,OK OK OK OK OK OK OK OK OK OK OK,Es diente als Veranstaltungsort für 2004 Pekan...,90.666667,0.433228,NaN,888,GPT4 Correct User: There is an English to Germ...,1. Accuracy:\n - Machine translation: Es die...,GPT4 Correct User: There is an English to Germ...,100
889,She retained her dowry and any settlement deed...,Sie behielt ihre Mitgift und jede Siedlung wur...,"Sie behielt ihre Mitgift und jede Einigung , d...",OK OK OK OK OK OK BAD BAD BAD OK OK BAD BAD OK OK,Sie behielt ihre Mitgift und jede Siedlung wur...,72.666667,-1.474221,"Siedlung, wurde, ihr, Mann, erlassen",889,GPT4 Correct User: There is an English to Germ...,1. Accuracy:\n - Machine translation: The ma...,GPT4 Correct User: There is an English to Germ...,90
890,Wolves typically kill moose by tearing at thei...,Wölfe töten Elche in der Regel durch Risse an ...,Wölfe töten Elche in der Regel durch Reißen an...,OK OK OK OK OK OK OK BAD OK OK BAD OK BAD OK O...,Wölfe töten Elche in der Regel durch Risse

In [6]:
df_en_de_test[['score', 'second_prompt_output']].describe()

,score,second_prompt_output
count,892.000000,892.000000
mean,85.448991,91.772422
std,11.998251,7.643194
min,1.000000,50.000000
25%,82.583333,90.000000
50%,89.000000,95.000000
75%,92.666667,95.000000
max,100.000000,100.000000


In [7]:
df_en_de_test.loc[df_en_de_test['score']>80][['score', 'second_prompt_output']].describe()

,score,second_prompt_output
count,701.000000,701.000000
mean,90.568949,92.348074
std,4.621061,7.316817
min,80.166667,50.000000
25%,87.333333,90.000000
50%,90.833333,95.000000
75%,93.666667,98.000000
max,100.000000,100.000000
